In [10]:
from cloudvolume import CloudVolume
from cloudvolume.exceptions import SkeletonDecodeError
from itertools import islice
from skimage.transform import downscale_local_mean
import napari
from skimage import io
import random
import h5py
from skimage import measure
from brainlit.preprocessing import removeSmallCCs
import numpy as np 
import matplotlib.pyplot as plt 
import subprocess
import tables
from napari_animation import AnimationWidget
from tqdm import tqdm
import pickle
import networkx as nx
import seaborn as sns
import pandas as pd
import brainrender
from brainlit.utils.session import NeuroglancerSession
from skimage.filters import threshold_otsu, threshold_local
from brainlit.preprocessing import split_frags, rename_states_consecutively, label_points
from brainlit.algorithms.connect_fragments import most_probable_neuron_path
from brainlit.algorithms.connect_fragments import trace_evaluation
from brainlit.algorithms.connect_fragments import dynamic_programming_viterbi2
import similaritymeasures
from brainlit.viz import Bresenham3D
from cloudvolume import Skeleton
from sklearn.metrics import pairwise_distances_argmin_min
import time
import zarr
from brainlit.preprocessing import image_process
%gui qt5

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

res = [0.3,0.3,1]
threshold = 0.9

block_size = 101

img, bbox, vox = ngl_sess.pull_vertex_list(2, [0], [1,1,1])
bbox

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"

vol = CloudVolume(dir, parallel=1, mip=0, fill_missing=True)
vol.shape

In [ ]:
#view
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_points(vox, size=7)
napari.run()

In [ ]:
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/500.tif", img)

In [ ]:
pred_path ="/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/250_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

mask = im_processed >= threshold
labels = measure.label(mask)

soma_coords = vox[0:1,:]

In [ ]:
new_labels = split_frags(soma_coords, labels, im_processed, threshold, res)
_, soma_lbls = label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]

In [ ]:
viterbi = most_probable_neuron_path(image=img.astype(float), labels=new_labels, soma_lbls=[soma_lbl], resolution=res, coef_dist=10, coef_curv=1000)
viterbi.frags_to_lines()
viterbi.reset_dists(type="all")
viterbi.compute_all_costs_dist(point_point_func=viterbi.point_point_dist, point_blob_func=viterbi.point_blob_dist)
viterbi.compute_all_costs_int()
viterbi.create_nx_graph()

In [ ]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/viterbi_250.pickle', 'wb') as handle:
    pickle.dump(viterbi, handle)

In [ ]:
from brainlit.utils.Neuron_trace import NeuronTrace

nt = NeuronTrace(path="/Users/thomasathey/Documents/mimlab/mouselight/input/mim_slides/SNT_Data.swc")
df = nt.get_df()
df[['x','y','z']].to_numpy()

In [2]:
im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/500.tif")
im2 = im.astype("float64")

In [ ]:
z = zarr.zeros(im.shape, chunks=(100,100,100), dtype='uint16')

In [ ]:
z[:,:,:] = im

In [ ]:
zarr.save("/Users/thomasathey/Documents/mimlab/mouselight/input/images/big/500.zarr", z)

In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(z, name="image", scale=[0.3,0.3,1])

In [3]:
from skimage.transform import pyramid_gaussian
multiscale = list(pyramid_gaussian(im2, downscale=2, max_layer=4))
print('multiscale level shapes: ', [p.shape for p in multiscale])
print(f"Original dtype: {im2.dtype}")
print(f"New dtype: {multiscale[0].dtype}")

multiscale level shapes:  [(1001, 1001, 1001), (501, 501, 501), (251, 251, 251), (126, 126, 126), (63, 63, 63)]
Original dtype: float64
New dtype: float64


In [5]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(multiscale, rgb=False, multiscale=True, name="multiscale image", scale=[0.3,0.3,1])
viewer.add_image(multiscale[0], rgb=False, name="image", scale=[0.3,0.3,1])

<Image layer 'image' at 0x1530b69a0>

Traceback (most recent call last):
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/utils/action_manager.py", line 53, in <lambda>
    self._command_with_context = lambda: call_with_context(
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/utils/action_manager.py", line 42, in call_with_context
    return function(**ctx)
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/components/_viewer_key_bindings.py", line 46, in toggle_ndisplay
    viewer.dims.ndisplay = 2
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/utils/events/evented_model.py", line 151, in __setattr__
    getattr(self.events, name)(value=after)  # emit event
  File "/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/utils/events/event.py", line 599, in __call__
    self._invoke_callback

## Make viterbi object

In [2]:
im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_0_first10_quantitative.tif"
probs_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/2018-08-01_0_first10_quantitative_Probabilities.h5"
res = [0.3,0.3,1]

im_og = io.imread(im_path, plugin="tifffile")

f = h5py.File(probs_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

threshold = 0.9  # 0.1
labels = measure.label(im_processed > threshold)

In [14]:
viewer = napari.Viewer(ndisplay=3)

viewer.add_image(im_og)
viewer.add_labels(labels)
viewer.add_points(soma_coords, face_color='red')



<Points layer 'Points' at 0x16abf23d0>

In [13]:
soma_coords = [[366,237, 122], [234,462, 122], [227, 641, 92]]

In [15]:
new_labels = image_process.compute_frags(soma_coords, labels, im_processed, threshold, res) #, chunk_size=[200,200,200], ncpu=2)

_, soma_lbls = image_process.label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]
soma_mask = new_labels == soma_lbl

remove small fragments: 100%|██████████| 2575/2575 [00:43<00:00, 58.53it/s] 


In [17]:
viewer = napari.Viewer(ndisplay=3)

viewer.add_image(im_og)
viewer.add_labels(labels)
viewer.add_labels(new_labels)
viewer.add_labels(soma_mask)

<Labels layer 'Labels' at 0x169b14700>

In [18]:
mpnp = dynamic_programming_viterbi2.most_probable_neuron_path(image=im_og.astype(float), labels=new_labels, soma_lbls=soma_lbls, resolution=(0.3, 0.3, 1), coef_dist=10, coef_curv=1000)#, parallel=2)
mpnp.compute_states()
mpnp.reset_dists(type="all")
mpnp.compute_all_costs_dist(point_point_func=mpnp.point_point_dist, point_blob_func=mpnp.point_blob_dist)
mpnp.compute_all_costs_int()
mpnp.create_nx_graph()

Setting up emission distribution...


Adding edges to nx graph: 100%|██████████| 3095/3095 [00:19<00:00, 156.45it/s]


In [19]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/viterbi_0_multisoma.pickle', 'wb') as handle:
    pickle.dump(mpnp, handle)